In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gender-recognizer/dataset/MEN/20240705_112131.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_133142.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_160304.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240609_200332.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_160729.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_114708.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_112707.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_101942.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_154853.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240609_183952.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_111531.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_160616.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240609_202926.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_115955.jpg
/kaggle/input/gender-recognizer/dataset/MEN/20240705_122708.jpg
/kaggle/input/gender-recognizer/dataset/

In [41]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder 
from torchvision import transforms 

In [3]:
dataset_loc = "/kaggle/input/gender-recognizer/dataset"

In [4]:
os.listdir(dataset_loc)

['MEN', 'WOMAN']

In [5]:
men_data = os.path.join(dataset_loc, "MEN")
women_data = os.path.join(dataset_loc, "WOMAN")

In [6]:
data_df = pd.DataFrame(columns=["filename", "label"])
for d in tqdm(os.listdir(men_data)):
    df = pd.DataFrame({"filename":os.path.join(dataset_loc, d), "label":"MEN"}, index=[0])
    data_df = pd.concat([data_df, df], ignore_index=True)

100%|██████████| 646/646 [00:00<00:00, 2021.40it/s]


In [7]:
for d in tqdm(os.listdir(women_data)):
    df = pd.DataFrame({"filename":os.path.join(dataset_loc, d), "label":"WOMAN"}, index=[0])
    data_df = pd.concat([data_df, df], ignore_index=True)

100%|██████████| 647/647 [00:00<00:00, 2073.44it/s]


In [8]:
data_df.head()

,filename,label
0,/kaggle/input/gender-recognizer/dataset/202407...,MEN
1,/kaggle/input/gender-recognizer/dataset/202407...,MEN
2,/kaggle/input/gender-recognizer/dataset/202407...,MEN
3,/kaggle/input/gender-recognizer/dataset/202406...,MEN
4,/kaggle/input/gender-recognizer/dataset/202407...,MEN


In [9]:
data_df.tail()

,filename,label
1288,/kaggle/input/gender-recognizer/dataset/0001.jpg,WOMAN
1289,/kaggle/input/gender-recognizer/dataset/0492.jpg,WOMAN
1290,/kaggle/input/gender-recognizer/dataset/0176.jpg,WOMAN
1291,/kaggle/input/gender-recognizer/dataset/0391.jpg,WOMAN
1292,/kaggle/input/gender-recognizer/dataset/0269.jpg,WOMAN


In [10]:
data_df=data_df.sample(frac=1.0).reset_index(drop=True)

In [11]:
data_df.head()

,filename,label
0,/kaggle/input/gender-recognizer/dataset/202407...,MEN
1,/kaggle/input/gender-recognizer/dataset/202406...,MEN
2,/kaggle/input/gender-recognizer/dataset/0193.jpg,WOMAN
3,/kaggle/input/gender-recognizer/dataset/202407...,MEN
4,/kaggle/input/gender-recognizer/dataset/0300.jpg,WOMAN


In [12]:
data_df.tail()

,filename,label
1288,/kaggle/input/gender-recognizer/dataset/Defaul...,WOMAN
1289,/kaggle/input/gender-recognizer/dataset/202406...,MEN
1290,/kaggle/input/gender-recognizer/dataset/tengra...,WOMAN
1291,/kaggle/input/gender-recognizer/dataset/202407...,MEN
1292,/kaggle/input/gender-recognizer/dataset/0034.jpg,WOMAN


In [30]:
for i in data_df["filename"]:
    if  (not i.endswith(".jpg")) and (not i.endswith(".png")):
        print(i)

/kaggle/input/gender-recognizer/dataset/MEN
/kaggle/input/gender-recognizer/dataset/6c6dcc79cb44204e1eca93838d2ab4eb6c2f027a_high.webp


In [36]:
data_df = data_df[data_df["filename"].str.endswith((".jpg", ".png"))]

In [37]:
class GenderRecognitionDataset(Dataset):
    def __init__(self, dataset_df):
        self.dataset_df = dataset_df
        self.filenames = self.dataset_df["filename"].to_list()
        self.labels = self.dataset_df["label"].to_list()
        self.classes = sorted(list(dataset_df['label'].unique()))
        self.class_to_idx = {cls_name: _ for _, cls_name in enumerate(self.classes)}
        
    def __len__(self):
        return len(self.dataset_df)
    
    def __getitem__(self, idx):
        image = read_image(self.filenames[idx])
        self.label_name = self.labels[idx]
        class_idx = self.class_to_idx[self.label_name]
        return image, class_idx
    

In [38]:
custom_data = GenderRecognitionDataset(data_df)

In [39]:
train_dataloader = DataLoader(custom_data, batch_size=4, shuffle=True)

In [40]:
# for images, labels in train_dataloader:
#     print(images, labels)

RuntimeError: [Errno 2] No such file or directory: '/kaggle/input/gender-recognizer/dataset/0229f8fa-8afc-494a-b1fb-fb3db88d546f.jpg'

In [45]:
train_transforms = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Resize((128, 128)), 
]) 

dataset_train = ImageFolder( 
    dataset_loc,  
    transform=train_transforms,
)

In [46]:


dataloader_train = DataLoader( 
     dataset_train,  
     shuffle=True, 
     batch_size=1, 
 ) 

In [47]:
image, label = next(iter(dataloader_train)) 

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
